In [1]:
!rm -rf .models
!mkdir .models

In [2]:
!wget -q -O .models/cifar_nat.pt "https://www.dropbox.com/s/yhpp4yws7sgi6lj/cifar_nat.pt?dl=1"
!wget -q -O .models/cifar_linf_8.pt "https://www.dropbox.com/s/c9qlt1lbdnu9tlo/cifar_linf_8.pt?dl=1"

In [3]:
import os
import pathlib
import random
import functools
import copy
import sys
import pickle
import tarfile
import operator
import math
import requests
import importlib
os.environ["NOTEBOOK_MODE"] = "1"

import tqdm
import cox

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import sklearn
from sklearn import preprocessing
import scipy

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

import torch

import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

import flax
import neural_tangents as nt
import art
import robustness
from robustness import model_utils, datasets, train, defaults

import vit_jax

import adversarial_ntks as atk
from adversarial_ntks import dataset

c:\users\0\main\active\monochrome\adversarial-ntks\robustness\robustness\train.py:24: UserWarning: Could not import amp.
  warnings.warn('Could not import amp.')


In [4]:
CIFAR10 = {}

In [5]:
CIFAR10.update({
    "train": atk.dataset.get_np_data(
        name="cifar10",
        split="train[:64]",
        flatten=False,
        data_dir=".data/"),
    "test": atk.dataset.get_np_data(
        name="cifar10",
        split="test[:256]",
        flatten=False,
        data_dir=".data/"),
    "dataset": robustness.datasets.CIFAR(".data"),
    "workers": 12,
    "arch": "resnet50",
    "resume_path_nat": ".models/cifar_nat.pt",
    "resume_path_linf": ".models/cifar_linf_8.pt",
    "labels": ["airplane", "automobile", "bird", "cat", "deer",
        "dog", "frog", "horse", "ship", "truck"],
    
    "batch_size": 256,
    "perturb_eps": 8/255,

    "display_samples": 10,
    "display_sample_img_size": 3})

ples [01:00, 736.42 examples/s]
Generating train examples...: 42100 examples [01:00, 766.56 examples/s]
Generating train examples...: 42183 examples [01:00, 784.91 examples/s]
Generating train examples...: 42262 examples [01:00, 758.28 examples/s]
Generating train examples...: 42339 examples [01:00, 724.52 examples/s]
Generating train examples...: 42412 examples [01:00, 688.81 examples/s]
Generating train examples...: 42493 examples [01:00, 721.89 examples/s]
Generating train examples...: 42567 examples [01:01, 724.58 examples/s]
Generating train examples...: 42641 examples [01:01, 728.34 examples/s]
Generating train examples...: 42716 examples [01:01, 734.25 examples/s]
Generating train examples...: 42799 examples [01:01, 761.94 examples/s]
Generating train examples...: 42876 examples [01:01, 742.18 examples/s]
Generating train examples...: 42951 examples [01:01, 739.23 examples/s]
Generating train examples...: 43026 examples [01:01, 733.40 examples/s]
Generating train examples...: 43

In [8]:
def exp_load_saved(exp):
    exp["model_nat"], _ = model_utils.make_and_restore_model(
        arch=exp["arch"],
        resume_path=exp["resume_path_nat"],
        dataset=exp["dataset"])
    exp["model_linf"], _ = model_utils.make_and_restore_model(
        arch=exp["arch"],
        resume_path=exp["resume_path_linf"],
        dataset=exp["dataset"])

exp_load_saved(CIFAR10)

=> loading checkpoint '.models/cifar_nat.pt'


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
def exp_load_saved(exp):
    exp["model_nat"], _ = model_utils.make_and_restore_model(
        arch=exp["arch"],
        resume_path=exp["resume_path_nat"],
        dataset=exp["dataset"])
    exp["model_linf"], _ = model_utils.make_and_restore_model(
        arch=exp["arch"],
        resume_path=exp["resume_path_linf"],
        dataset=exp["dataset"])

exp_load_saved(CIFAR10)

In [ ]:
def exp_eval_ueb_atk(exp, models, dss, samples):
    c_correct = {}
    for ds in dss:
        xs_gpu = torch.Tensor(np.transpose(exp[ds].xs, (0, 3, 1, 2))).cuda()
        ys_gpu = torch.Tensor(exp[ds].ys).cuda()
        c_batches = len(exp[ds].xs) // exp["batch_size"]

        c_correct[ds] = {}
        for model in models:
            c_correct[ds][model] = []
    
        for batch_id in tqdm.notebook.tqdm(range(c_batches), desc="Batch"):
            batch_start = batch_id * exp["batch_size"]
            batch_end = batch_start + exp["batch_size"]
            batch_xs = xs_gpu[batch_start:batch_end]
            batch_ys = ys_gpu[batch_start:batch_end]

            c_correct_batch = {}
            for model in models:
                c_correct_batch[model] = []

            for sample in tqdm.notebook.tqdm(range(samples), desc="Sample"):
                xs_perturbation = (torch.rand(batch_xs.shape) * \
                    exp["perturb_eps"] * 2 - exp["perturb_eps"]).cuda()
                xs_perturbed = batch_xs.add(xs_perturbation).clamp(0, 1)

                for model in models:
                    predictions = exp[model](xs_perturbed)[0].argmax(dim=1)
                    c_correct_batch[model].append(
                        predictions.eq(batch_ys).cpu())
                        
                    del predictions

                del xs_perturbation, xs_perturbed

            for model in models:
                c_correct[ds][model].append(np.vstack(
                    c_correct_batch[model]).transpose())

        for model in models:
            c_correct[ds][model] = \
                np.concatenate(c_correct[ds][model]).astype(int)

        del xs_gpu, ys_gpu
    
    return c_correct

CIFAR10["ueb-atk-res"] = exp_eval_ueb_atk(
    CIFAR10,
    ["model_nat", "model_linf"],
    ["test"],
    512)

In [ ]:
def exp_show_ueb_atk_res(exp):
    res = CIFAR10["ueb-atk-res"]
    dss, models = [], []
    for ds in res.keys():
        dss.append(ds)
        if len(models) == 0:
            for model in res[ds].keys():
                models.append(model)

    print(dss, models)
    fig, ax = plt.subplots(len(dss), len(models), figsize=(5 * len(models), 5 * len(dss)))
    fig2, ax2 = plt.subplots(1, len(dss), figsize=(5 * len(dss), 5))
    fig3, ax3 = plt.subplots(1, len(dss), figsize=(5 * len(dss), 5))

    for ds_i in range(len(dss)):
        ds = dss[ds_i]
        for model_i in range(len(models)):
            model = models[model_i]
            cur_ax = ax if type(ax) is not np.ndarray else (
                ax[model_i] if type(ax[model_i]) is not np.ndarray else
                    ax[ds_i][model_i])
            uebp = 1 - \
                np.sum(res[ds][model], axis=1) / res[ds][model].shape[1]
            cur_ax.hist(uebp, bins=64, range=(0, 1))
            cur_ax.set_title(ds + " // " + model + " // {:.4f}".format(
                np.count_nonzero(1 - res[ds][model].mean(axis=1)) /
                res[ds][model].shape[0]))
            cur_ax.set_xlabel("% of adversarial samples")
            cur_ax.set_ylabel("Number of points")

        if len(models) == 2:
            cur_ax = ax2 if type(ax2) is not np.ndarray else ax2[ds_i]
            res1, res2 = res[ds][models[0]], res[ds][models[1]]
            eval_equal = res1 == res2
            overlap = np.sum(eval_equal, axis=1) / res1.shape[1]
            cur_ax.hist(overlap, bins=64, range=(0, 1))
            cur_ax.set_title(ds + " // {:.4f}".format(overlap.mean()))
            cur_ax.set_xlabel("% of equally adversarial samples b/t models")
            cur_ax.set_ylabel("Number of points")

            cur_ax = ax3 if type(ax3) is not np.ndarray else ax3[ds_i]
            res_means = 1 - res1.mean(axis=1), 1 - res2.mean(axis=1)
            cur_ax.scatter(*res_means)
            cur_ax.set_title(ds)
            cur_ax.set_xlabel(models[0] + " uebp")
            cur_ax.set_ylabel(models[1] + " uebp")
            cur_ax.set_xlim(-0.1, 1.1)
            cur_ax.set_ylim(-0.1, 1.1)

    fig.show()
    fig2.show()
    fig3.show()

exp_show_ueb_atk_res(CIFAR10)